# Preprocessing the Data

In [29]:
from tqdm import tqdm_notebook as tqdm
import preprocessor as p
import numpy as np
import json

In [30]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torch.nn.functional as F

In [31]:
from collections import Counter
from tqdm import tqdm, tqdm_notebook, tnrange
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [32]:
import matplotlib.pyplot as plt
import random
import pickle as pkl

In [34]:
dataset = 'twitter16'
    
if dataset == 'twitter15':
    %run ../twitter15/twitter15_text_processing.ipynb
    with open('./kfolds_twitter15.pickle','rb') as f:
        kfold_df = pkl.load(f)
    datadir = './twitter15_kfold_results/'
    
if dataset == 'twitter16':
    %run ../twitter16/twitter16_text_processing.ipynb
    with open('./kfolds_twitter16.pickle','rb') as f:
        kfold_df = pkl.load(f)
    datadir = './twitter16_kfold_results/'

In [35]:
def preprocess(text):
    p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.EMOJI,p.OPT.HASHTAG)
    return p.tokenize(text).split()

In [36]:
twitter15_text

{656955120626880512: 'correct predictions in back to the future ii URL',
 615689290706595840: ".@whitehouse in rainbow colors for #scotusmarriage? here's what i think about that decision: URL",
 613404935003217920: 'cops bought the alleged church shooter burger king hours after killings URL',
 731166399389962242: '🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL',
 714598641827246081: 'an open letter to trump voters from his top strategist-turned-defector URL via @xojanedotcom',
 614467824313106432: 'god put a rainbow over the white house 🌈 URL',
 715515982584881152: 'craigslist ad: ‘get paid $15 an hour to protest at the trump rally’ - URL URL',
 693315824132685824: '#quasimodo: dog with short spine has big heart URL URL',
 693843042546106369: 'need to #getcovered? most people can find a health insurance plan for as little as $75/mo: URL URL',
 622891631293935616: '#wakeupamerica🇺🇸 who needs a #gun registry when #obama has all your personal information URL 

preprocess(twitter15_text[656955120626880512])

In [37]:
twitter15_text

{656955120626880512: 'correct predictions in back to the future ii URL',
 615689290706595840: ".@whitehouse in rainbow colors for #scotusmarriage? here's what i think about that decision: URL",
 613404935003217920: 'cops bought the alleged church shooter burger king hours after killings URL',
 731166399389962242: '🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL',
 714598641827246081: 'an open letter to trump voters from his top strategist-turned-defector URL via @xojanedotcom',
 614467824313106432: 'god put a rainbow over the white house 🌈 URL',
 715515982584881152: 'craigslist ad: ‘get paid $15 an hour to protest at the trump rally’ - URL URL',
 693315824132685824: '#quasimodo: dog with short spine has big heart URL URL',
 693843042546106369: 'need to #getcovered? most people can find a health insurance plan for as little as $75/mo: URL URL',
 622891631293935616: '#wakeupamerica🇺🇸 who needs a #gun registry when #obama has all your personal information URL 

In [38]:
twitter15_labels

{656955120626880512: 'false',
 615689290706595840: 'true',
 613404935003217920: 'false',
 731166399389962242: 'unverified',
 714598641827246081: 'unverified',
 614467824313106432: 'true',
 715515982584881152: 'unverified',
 693315824132685824: 'non-rumor',
 693843042546106369: 'non-rumor',
 622891631293935616: 'false',
 692630756548591616: 'non-rumor',
 693265096278163456: 'non-rumor',
 553589051044151296: 'true',
 553590835850514433: 'true',
 622858454949040128: 'false',
 656870311057575936: 'false',
 616311563071434753: 'true',
 641666167992647681: 'non-rumor',
 525060425184858112: 'true',
 672513234419638273: 'false',
 544382892378714113: 'true',
 681824512120324096: 'non-rumor',
 620835698514464768: 'false',
 626898253900943360: 'false',
 618804516578680832: 'false',
 672632899921833984: 'false',
 553588178687655936: 'true',
 594687353937100801: 'false',
 613016993692798977: 'false',
 663385747177775105: 'false',
 766715993385267201: 'non-rumor',
 693141729529184256: 'non-rumor',
 

In [39]:
len(twitter15_text)

818

# Text Only Baseline


In [40]:
def preprocess(text):
    p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI ,p.OPT.HASHTAG)
    return p.tokenize(text).split()

In [41]:
def indexer(split_text):
    sent2idx = []
    for w in split_text:
        if w.lower() in word2idx:
            sent2idx.append(word2idx[w.lower()])
        else:
            sent2idx.append(word2idx['_UNK'])
            
    return sent2idx

In [42]:
def trainModelWithKfold(foldnum,datadir):
    '''
    STEP 1: LOADING DATASET
    '''

    labels = {'true':0,'false':1,'unverified':2,'non-rumor':3}

    data = list(twitter15_text.keys())

    train_ids, valid_ids = kfold_df[foldnum]

    train_text = [twitter15_text[x] for x in train_ids]
    valid_text = [twitter15_text[x] for x in valid_ids]

    train = pd.DataFrame({'text_id':train_ids ,'raw_text':train_text})
    train['clean_text'] = train.raw_text.apply(lambda x: preprocess(x.strip()))

    valid = pd.DataFrame({'text_id':valid_ids ,'raw_text':valid_text})
    valid['clean_text'] = valid.raw_text.apply(lambda x: preprocess(x.strip()))

    words = Counter()
    for sent in tqdm(train.clean_text.values):
        words.update(w.lower() for w in sent)

    for sent in tqdm(valid.clean_text.values):
        words.update(w.lower() for w in sent)

    # sort with most frequently occuring words first
    words = sorted(words, key=words.get, reverse=True)
    # add <pad> and <unk> token to vocab which will be used later
    words = ['_PAD','_UNK'] + words
    print(len(words))

    global word2idx
    word2idx = {o:i for i,o in enumerate(words)}
    idx2word = {i:o for i,o in enumerate(words)}

    train['sentence2idx'] = train.clean_text.apply(lambda x: indexer(x))
    train['length'] = train.clean_text.apply(lambda x: len(x))
    train['label'] = train.text_id.apply(lambda x: labels[twitter15_labels[x]])

    valid['sentence2idx'] = valid.clean_text.apply(lambda x: indexer(x))
    valid['length'] = valid.clean_text.apply(lambda x: len(x))
    valid['label'] = valid.text_id.apply(lambda x: labels[twitter15_labels[x]])
    
    train_loader = VectorizeData(train)
    valid_loader = VectorizeData(valid)
    
    global tl,vl
    tl = DataLoader(dataset=train_loader, batch_size=100, shuffle=True)
    vl = DataLoader(dataset=valid_loader, batch_size=100, shuffle=False)

    vocab_size = len(words)
    embedding_dim = 256
    n_hidden = 50
    n_out = 4
    num_layers = 1
    rnnType = 'gru'

    # n_hiddens = [5,25,50,100,200]
    # n_layers = [1,2,3]
    n_hiddens = [50]
    n_layers = [1]

    rnnTypes = ['gru','lstm']
    bidirTypes = [False,True]

    if torch.cuda.is_available():
        device = 'cuda:1'
    else:
        device = 'cpu'

    results = {}

#     print(vocab_size)
    for bidirType in bidirTypes:
        for n_hidden in n_hiddens:
            for num_layers in n_layers:
                for rnnType in rnnTypes:
                    key = (n_hidden,num_layers,rnnType,bidirType)
    #                 acc,f1 = train(vocab_size,embedding_dim,n_hidden,n_out,num_layers,bidirType,rnnType,device,0)
                    cr = trainModel(vocab_size,embedding_dim,n_hidden,n_out,num_layers,bidirType,rnnType,device,1,foldnum)
                    with open(datadir+cr['modelname']+'.json', 'a') as fp:
                        json.dump(cr, fp)
                        fp.write('\n')

#                     results[key] = (acc,f1)

    results
    return results

In [43]:
class VectorizeData(Dataset):
    def __init__(self, df, maxlen=30):
        self.maxlen = maxlen
        self.df = df
#         print('Padding')
        self.df['padded_text'] = self.df.sentence2idx.apply(lambda x: self.pad_data(x))
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
#         lens = self.df.length[idx]
        X = self.df.padded_text[idx]
        y = self.df.label[idx]
        lens = self.df.length[idx]
        return X,y,lens
    
    def pad_data(self, s):
        padded = np.zeros((self.maxlen,), dtype=np.int64)
        if len(s) > self.maxlen: padded[:] = s[:self.maxlen]
        else: padded[:len(s)] = s
        return padded

## RecModel and RecAttnArchitecture

In [44]:
class RecArch(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidir, rnnType,device):
        super(RecArch, self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.device = device
        self.rnnType = rnnType
        self.bidirectional = bidir
        
        if self.bidirectional:
            self.numDirs = 2
        else:
            self.numDirs = 1
        
        self.emb = nn.Embedding(self.vocab_size, embedding_dim)
        
        if self.rnnType == 'lstm':
            self.recNN = nn.LSTM(embedding_dim,hidden_dim, num_layers,batch_first=True,bidirectional=self.bidirectional)
            
        if self.rnnType == 'gru':
            self.recNN = nn.GRU(embedding_dim, hidden_dim, num_layers, batch_first=True,bidirectional=self.bidirectional)
            
        if self.rnnType == 'rnn':
            self.recNN = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True, nonlinearity='tanh',bidirectional=self.bidirectional)
        
        self.fc = nn.Linear(self.numDirs*hidden_dim,output_dim)
    
    def forward(self,x):
        embs = self.emb(x)
        embs = embs.view(x.size(0),-1,self.embedding_dim).to(self.device)
        
        h0 = Variable(torch.zeros(self.numDirs*self.num_layers,x.size(0),self.hidden_dim),requires_grad=True).to(self.device)
        
        if self.rnnType == 'lstm':        
            c0 = Variable(torch.zeros(self.numDirs*self.num_layers,x.size(0),self.hidden_dim),requires_grad=True).to(self.device)
            
            out,(hn,cn) = self.recNN(embs,(h0,c0))
        
        else:
            out, hn = self.recNN(embs, h0)
        
#         print(out[:,-1,:].shape)
        out = self.fc(out[:, -1, :])
        return out

In [45]:
class RecAttnArch(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidir, rnnType, attnType,device):
        super(RecAttnArch, self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.device = device
        self.rnnType = rnnType
        self.attnType = attnType
        self.bidirectional = bidir
        
        if self.bidirectional:
            self.numDirs = 2
        else:
            self.numDirs = 1
        
        self.emb = nn.Embedding(self.vocab_size, embedding_dim)
        
        if self.rnnType == 'lstm':
            self.recNN = nn.LSTM(embedding_dim,hidden_dim, num_layers,batch_first=True,bidirectional=self.bidirectional)
            
        if self.rnnType == 'gru':
            self.recNN = nn.GRU(embedding_dim, hidden_dim, num_layers, batch_first=True,bidirectional=self.bidirectional)
            
        if self.rnnType == 'rnn':
            self.recNN = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True, nonlinearity='tanh',bidirectional=self.bidirectional)
        
        self.query_vector = nn.Parameter(torch.rand(hidden_dim*self.numDirs,1)).float()
        
        self.attnWgtMatrixSize = [self.numDirs*self.hidden_dim, self.numDirs*self.hidden_dim]
        self.attnWgtMatrix = nn.Parameter(torch.randn(self.attnWgtMatrixSize).float()) # Multiplicative Attention
    
        self.softmax = nn.Softmax(dim=1)
        
        if self.attnType == 'dot':
            self.fc = nn.Linear(self.numDirs*self.hidden_dim, output_dim)
        
        if self.attnType == 'self':
            self.fc = nn.Linear(self.numDirs*30*self.hidden_dim, output_dim)
    
    
    def forward(self,x,encMode=False):
        embs = self.emb(x)
        embs = embs.view(x.size(0),-1,self.embedding_dim).to(self.device)
        
        h0 = Variable(torch.zeros(self.numDirs*self.num_layers,x.size(0),self.hidden_dim),requires_grad=True).to(self.device)
        
        if self.rnnType == 'lstm':        
            c0 = Variable(torch.zeros(self.numDirs*self.num_layers,x.size(0),self.hidden_dim),requires_grad=True).to(self.device)
            
            out,(hn,cn) = self.recNN(embs,(h0,c0))
        
        else:
            out, hn = self.recNN(embs, h0)
        
        if self.attnType == 'dot':
            Hw = out
            attn_weights = self.softmax(Hw.matmul(self.query_vector))

            out = out.mul(attn_weights)
            context_vector = torch.sum(out,dim=1)
            
            fc_out = context_vector
            
        if self.attnType == 'self':
            queryMatrix = out
            keyMatrix = out.permute(0,2,1)
            
            attnScores = torch.bmm( torch.matmul(queryMatrix,self.attnWgtMatrix), keyMatrix )
            attnScores = F.softmax(attnScores, dim=2)
            hidden_matrix = torch.bmm(attnScores, queryMatrix)
            
            fc_out = hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2])
        
        if encMode:
            return fc_out
            
        else:
            return self.fc(fc_out)

#### Baseline

vocab_size = len(words)
embedding_dim = 256
n_hidden = 50
n_out = 4
num_layers = 1
rnnType = 'gru'
bidir = True

if torch.cuda.is_available():
    device = 'cuda:2'
    
else:
    device = 'cpu'
    
model = RecAttnArch(vocab_size,embedding_dim,n_hidden,n_out,num_layers,bidir,rnnType,device)
model.to(device)
model.float()

optimizer = torch.optim.Adagrad(model.parameters(),lr=0.01)
# criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss()

count = 0
seq_dim = 30
num_epochs = 200

train_losses_iterwise = []
recall_iterwise = []
precision_iterwise = []
accuracy_iterwise = []
f1score_iterwise = []
val_losses_iterwise = []

for epoch in tqdm(range(num_epochs)):
    train_losses = []
    val_losses = []
    for i, (text,label,lengths) in enumerate(tl):

        text = Variable(text.view(-1, seq_dim, 1)).to(device)
        label = Variable(label).to(device)
        
#         print(sexism_label)
        
        optimizer.zero_grad()
        outputs = model(text)
        
#         print(outputs)
        
        loss = criterion(outputs, label)
        train_losses.append(loss.data.cpu())
        
        loss.backward()
        optimizer.step()
        count += 1
        
        if count % 50 == 0:    
            correct = 0
            total = 0

            allLabels = []
            allPreds = []
            
            for i, (text,label,lengths) in enumerate(vl):
                labels=[]
                text = Variable(text.view(-1, seq_dim, 1)).to(device)
                label = Variable(label).to(device)
                
                predicted = model(text)
                predicted =  torch.softmax(predicted,1)
                predicted = torch.max(predicted, 1)[1].cpu().numpy().tolist()
#                 print(predicted)
#                 print(sexism_label)
                allLabels += (label.cpu().numpy().tolist())
                allPreds += (predicted)

            valacc = accuracy_score(allLabels, allPreds)
            recscore = recall_score(allLabels, allPreds,average='macro')
            precscore = precision_score(allLabels, allPreds,average='macro')
            f1score = f1_score(allLabels, allPreds,average='macro')
            cr = classification_report(allLabels, allPreds)
            print(f'acc: {valacc} recall {recscore} prec: {precscore} f1: {f1score}')
            print(cr)
            
            train_losses_iterwise.append(np.mean(train_losses))

### Optimal combination seems to be with GRU of 50 units and 1 layer and LSTM of 200 units and 1 layer

vocab_size = len(words)
embedding_dim = 256
n_hidden = 100
n_out = 4
num_layers = 1
bidir = False
rnnType = 'gru'

if torch.cuda.is_available():
    device = 'cuda:1'
else:
    device = 'cpu'

model = RecArch(vocab_size,embedding_dim,n_hidden,n_out,num_layers,bidir,rnnType,device)
model.to(device)
model.float()

optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
# criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss()

count = 0
seq_dim = 30
num_epochs = 200

train_losses_iterwise = []
recall_iterwise = []
precision_iterwise = []
accuracy_iterwise = []
f1score_iterwise = []
val_losses_iterwise = []

for epoch in tqdm(range(num_epochs)):
    train_losses = []
    val_losses = []
    for i, (text,label,lengths) in enumerate(tl):
        text = Variable(text.view(-1, seq_dim, 1)).to(device)
        label = Variable(label).to(device)
        
#         print(sexism_label)
        
        optimizer.zero_grad()
        outputs = model(text)
        
#         print(outputs)
        
        loss = criterion(outputs, label)
        train_losses.append(loss.data.cpu())
        
        loss.backward()
        optimizer.step()
        count += 1
        
        if count % 50 == 0:    
            correct = 0
            total = 0

            allLabels = []
            allPreds = []
            
            for i, (text,label,lengths) in enumerate(vl):
                labels=[]
                text = Variable(text.view(-1, seq_dim, 1)).to(device)
                label = Variable(label).to(device)
                
                predicted = model(text)
                predicted =  torch.softmax(predicted,1)
                predicted = torch.max(predicted, 1)[1].cpu().numpy().tolist()
#                 print(predicted)
#                 print(sexism_label)
                allLabels += (label.cpu().numpy().tolist())
                allPreds += (predicted)

            valacc = accuracy_score(allLabels, allPreds)
            recscore = recall_score(allLabels, allPreds,average='macro')
            precscore = precision_score(allLabels, allPreds,average='macro')
            f1score = f1_score(allLabels, allPreds,average='macro')
            cr = classification_report(allLabels, allPreds)
            print(f'acc: {valacc} recall {recscore} prec: {precscore} f1: {f1score}')
            print(cr)
            
            train_losses_iterwise.append(np.mean(train_losses))

torch.save({'state_dict': model.state_dict()}, './gruTextEnc_twit16.pth')

vocab_size

In [48]:
def trainModel(vocab_size,embedding_dim,n_hidden,n_out,num_layers,bidir,rnnType,device,returncr,foldnum):
#     model = RecAttnArch(vocab_size,embedding_dim,n_hidden,n_out,num_layers,bidir,rnnType,'self',device)
    maxAcc = 0
    model = RecArch(vocab_size,embedding_dim,n_hidden,n_out,num_layers,bidir,rnnType,device)
    model.to(device)
    model.float()
    
    modelname = rnnType
#     modelname = rnnType + 'withAttention'
    if bidir:
        modelname = 'Bi'+modelname
    
    optimizer = torch.optim.Adagrad(model.parameters(),lr=0.01)
    # criterion = torch.nn.BCEWithLogitsLoss()
    criterion = torch.nn.CrossEntropyLoss()

    count = 0
    seq_dim = 30
    num_epochs = 300

    train_losses_iterwise = []
    recall_iterwise = []
    precision_iterwise = []
    accuracy_iterwise = []
    f1score_iterwise = []
    val_losses_iterwise = []

    for epoch in tqdm(range(num_epochs)):
        train_losses = []
        val_losses = []
        for i, (text,label,lengths) in enumerate(tl):

            text = Variable(text.view(-1, seq_dim, 1)).to(device)
            label = Variable(label).to(device)

    #         print(sexism_label)

            optimizer.zero_grad()
            outputs = model(text)

    #         print(outputs)

            loss = criterion(outputs, label)
            train_losses.append(loss.data.cpu())

            loss.backward()
            optimizer.step()
            count += 1

            if count % 100 == 0:    
                correct = 0
                total = 0

                allLabels = []
                allPreds = []

                for i, (text,label,lengths) in enumerate(vl):
                    labels=[]
                    text = Variable(text.view(-1, seq_dim, 1)).to(device)
                    label = Variable(label).to(device)

                    predicted = model(text)
                    predicted =  torch.softmax(predicted,1)
                    predicted = torch.max(predicted, 1)[1].cpu().numpy().tolist()
    #                 print(predicted)
    #                 print(sexism_label)
                    allLabels += (label.cpu().numpy().tolist())
                    allPreds += (predicted)

                valacc = accuracy_score(allLabels, allPreds)
                recscore = recall_score(allLabels, allPreds,average='macro')
                precscore = precision_score(allLabels, allPreds,average='macro')
                f1score = f1_score(allLabels, allPreds,average='macro')
                cr = classification_report(allLabels, allPreds,output_dict=True)
                cr['acc'] = valacc
                cr['modelname'] = modelname
                cr['fold'] = foldnum
                
                if cr['acc'] > maxAcc:
                    maxAcc = cr['acc']
                    print(cr['acc'])
                    print(vocab_size)
                         
                    torch.save({'state_dict': model.state_dict()}, './twitter15_kfold_results/'+modelname+'_'+str(foldnum)+'.pth')
                
                accuracy_iterwise.append(valacc)
                f1score_iterwise.append(f1score)
#                 print(f'acc: {valacc} recall {recscore} prec: {precscore} f1: {f1score}')
#                 print(cr)

                train_losses_iterwise.append(np.mean(train_losses))
    
    if returncr:
        print(modelname)
        print(cr)
        return cr
    
    return max(accuracy_iterwise),max(f1score_iterwise)

In [ ]:
for i in range(0,4):
    trainModelWithKfold(i,datadir)


3702


0.6195121951219512
3702
0.6390243902439025
3702
0.6536585365853659
3702

gru
{'0': {'precision': 0.639344262295082, 'recall': 0.8863636363636364, 'f1-score': 0.742857142857143, 'support': 44}, '1': {'precision': 0.7169811320754716, 'recall': 0.5135135135135135, 'f1-score': 0.5984251968503936, 'support': 74}, '2': {'precision': 0.6111111111111112, 'recall': 0.7674418604651163, 'f1-score': 0.6804123711340206, 'support': 43}, '3': {'precision': 0.4864864864864865, 'recall': 0.4090909090909091, 'f1-score': 0.44444444444444453, 'support': 44}, 'micro avg': {'precision': 0.624390243902439, 'recall': 0.624390243902439, 'f1-score': 0.624390243902439, 'support': 205}, 'macro avg': {'precision': 0.6134807479920378, 'recall': 0.6441024798582938, 'f1-score': 0.6165347888215005, 'support': 205}, 'weighted avg': {'precision': 0.6286387048670815, 'recall': 0.624390243902439, 'f1-score': 0.6135730066681067, 'support': 205}, 'acc': 0.624390243902439, 'modelname': 'gru', 'fold': 0}


0.375609756097561
3702
0.5756097560975609
3702
0.6097560975609756
3702
0.6487804878048781
3702
0.6585365853658537
3702
0.6634146341463415
3702
0.6682926829268293
3702
0.6731707317073171
3702

lstm
{'0': {'precision': 0.8048780487804879, 'recall': 0.75, 'f1-score': 0.7764705882352942, 'support': 44}, '1': {'precision': 0.7123287671232876, 'recall': 0.7027027027027027, 'f1-score': 0.7074829931972789, 'support': 74}, '2': {'precision': 0.6304347826086957, 'recall': 0.6744186046511628, 'f1-score': 0.651685393258427, 'support': 43}, '3': {'precision': 0.4666666666666667, 'recall': 0.4772727272727273, 'f1-score': 0.4719101123595506, 'support': 44}, 'micro avg': {'precision': 0.6585365853658537, 'recall': 0.6585365853658537, 'f1-score': 0.6585365853658537, 'support': 205}, 'macro avg': {'precision': 0.6535770662947845, 'recall': 0.6510985086566481, 'f1-score': 0.6518872717626376, 'support': 205}, 'weighted avg': {'precision': 0.662287765360839, 'recall': 0.6585365853658537, 'f1-score': 0.6600

0.6
3702
0.6097560975609756
3702
0.6146341463414634
3702
0.624390243902439
3702

Bigru
{'0': {'precision': 0.6923076923076923, 'recall': 0.8181818181818182, 'f1-score': 0.7500000000000001, 'support': 44}, '1': {'precision': 0.7142857142857143, 'recall': 0.6756756756756757, 'f1-score': 0.6944444444444444, 'support': 74}, '2': {'precision': 0.5384615384615384, 'recall': 0.4883720930232558, 'f1-score': 0.5121951219512196, 'support': 43}, '3': {'precision': 0.4772727272727273, 'recall': 0.4772727272727273, 'f1-score': 0.4772727272727273, 'support': 44}, 'micro avg': {'precision': 0.624390243902439, 'recall': 0.624390243902439, 'f1-score': 0.624390243902439, 'support': 205}, 'macro avg': {'precision': 0.605581918081918, 'recall': 0.6148755785383693, 'f1-score': 0.6084780734170978, 'support': 205}, 'weighted avg': {'precision': 0.6218172071830609, 'recall': 0.624390243902439, 'f1-score': 0.621528190891665, 'support': 205}, 'acc': 0.624390243902439, 'modelname': 'Bigru', 'fold': 0}


0.5365853658536586
3702
0.6146341463414634
3702

Bilstm
{'0': {'precision': 0.6875, 'recall': 0.75, 'f1-score': 0.717391304347826, 'support': 44}, '1': {'precision': 0.7377049180327869, 'recall': 0.6081081081081081, 'f1-score': 0.6666666666666667, 'support': 74}, '2': {'precision': 0.5777777777777777, 'recall': 0.6046511627906976, 'f1-score': 0.5909090909090908, 'support': 43}, '3': {'precision': 0.43137254901960786, 'recall': 0.5, 'f1-score': 0.4631578947368421, 'support': 44}, 'micro avg': {'precision': 0.6146341463414634, 'recall': 0.6146341463414634, 'f1-score': 0.6146341463414634, 'support': 205}, 'macro avg': {'precision': 0.6085888112075433, 'recall': 0.6156898177247014, 'f1-score': 0.6095312391651064, 'support': 205}, 'weighted avg': {'precision': 0.6276341489547972, 'recall': 0.6146341463414634, 'f1-score': 0.6179833609860959, 'support': 205}, 'acc': 0.6146341463414634, 'modelname': 'Bilstm', 'fold': 0}


...

0.5951219512195122
3702
0.6439024390243903
3702

lstm
{'0': {'precision': 0.6538461538461539, 'recall': 0.8095238095238095, 'f1-score': 0.7234042553191489, 'support': 42}, '1': {'precision': 0.7213114754098361, 'recall': 0.7096774193548387, 'f1-score': 0.7154471544715446, 'support': 62}, '2': {'precision': 0.6739130434782609, 'recall': 0.5961538461538461, 'f1-score': 0.6326530612244897, 'support': 52}, '3': {'precision': 0.5, 'recall': 0.46938775510204084, 'f1-score': 0.4842105263157895, 'support': 49}, 'micro avg': {'precision': 0.6439024390243903, 'recall': 0.6439024390243903, 'f1-score': 0.6439024390243903, 'support': 205}, 'macro avg': {'precision': 0.6372676681835627, 'recall': 0.6461857075336338, 'f1-score': 0.6389287493327432, 'support': 205}, 'weighted avg': {'precision': 0.6425674546235017, 'recall': 0.6439024390243903, 'f1-score': 0.6408047671892056, 'support': 205}, 'acc': 0.6439024390243903, 'modelname': 'lstm', 'fold': 1}


0.6195121951219512
3702

Bigru
{'0': {'precision': 0.5660377358490566, 'recall': 0.7142857142857143, 'f1-score': 0.631578947368421, 'support': 42}, '1': {'precision': 0.6266666666666667, 'recall': 0.7580645161290323, 'f1-score': 0.6861313868613139, 'support': 62}, '2': {'precision': 0.7045454545454546, 'recall': 0.5961538461538461, 'f1-score': 0.6458333333333334, 'support': 52}, '3': {'precision': 0.48484848484848486, 'recall': 0.32653061224489793, 'f1-score': 0.3902439024390244, 'support': 49}, 'micro avg': {'precision': 0.6048780487804878, 'recall': 0.6048780487804878, 'f1-score': 0.6048780487804878, 'support': 205}, 'macro avg': {'precision': 0.5955245854774157, 'recall': 0.5987586722033726, 'f1-score': 0.5884468925005232, 'support': 205}, 'weighted avg': {'precision': 0.6001017445508933, 'recall': 0.6048780487804878, 'f1-score': 0.5940085186718082, 'support': 205}, 'acc': 0.6048780487804878, 'modelname': 'Bigru', 'fold': 1}


0.40487804878048783
3702
0.45365853658536587
3702
0.5121951219512195
3702
0.5902439024390244
3702
0.6195121951219512
3702
0.6390243902439025
3702
0.6487804878048781
3702

Bilstm
{'0': {'precision': 0.9259259259259259, 'recall': 0.5952380952380952, 'f1-score': 0.7246376811594204, 'support': 42}, '1': {'precision': 0.6438356164383562, 'recall': 0.7580645161290323, 'f1-score': 0.6962962962962962, 'support': 62}, '2': {'precision': 0.5, 'recall': 0.8076923076923077, 'f1-score': 0.6176470588235294, 'support': 52}, '3': {'precision': 0.7142857142857143, 'recall': 0.30612244897959184, 'f1-score': 0.42857142857142855, 'support': 49}, 'micro avg': {'precision': 0.6292682926829268, 'recall': 0.6292682926829268, 'f1-score': 0.6292682926829268, 'support': 205}, 'macro avg': {'precision': 0.6960118141624991, 'recall': 0.6167793420097568, 'f1-score': 0.6167881162126686, 'support': 205}, 'weighted avg': {'precision': 0.681983888332034, 'recall': 0.6292682926829268, 'f1-score': 0.6181600001848271, 'su


3702


0.5980392156862745
3702
0.6078431372549019
3702
0.6274509803921569
3702
0.6372549019607843
3702
0.6421568627450981
3702
0.6470588235294118
3702
